In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # O '0,1' para usar múltiples GPUs
import tensorflow as tf
tf.test.gpu_device_name()

In [2]:
from PIL import Image
import numpy as np
from tensorflow.keras.utils import Sequence
from glob import glob

In [3]:
dataset = glob("dataset/*/*.jpg", recursive = True)
classes = list(set([k.split("/")[1] for k in dataset]))
classes

['test']

In [4]:
def encoder_class(classes, clasz):
    index = classes.index(clasz)
    result = np.zeros(len(classes))
    result[index] = 1
    return result

In [5]:
class Mygenerator(Sequence):
    def __init__(self, data, batch_size, readers, training = False):
        self.data = data
        self.batch_size = batch_size
        self.readers = readers
        self.training = training
    def __len__(self):
        return int(np.ceil(len(self.data)/float(self.batch_size)))
    
    def __getitem__(self,idx):
        batch = self.data[ idx * self.batch_size: (idx + 1) * self.batch_size]

        x = []
        y = []

        for filename in batch:
            x_, y_ = self.readers(filename,self.training)
            x.append(x_)
            y.append(y_)
        return np.array(x), np.array(y)

In [6]:
def wrapper_read_x_y(classes):
    def read_x_y(filename, training = False):
        img = Image.open(filename).convert("RGB")
        img2 = img.resize((224,224))
        
        label = encoder_class(classes,filename.split("/")[1])

        if not training:
            return np.array(img2) / 255, label

        width, height = img2.size
        x_range, y_range = int(width * 0.3), int(height * 0.3)
        random_x = np.random.randint(0,x_range)
        random_y = np.random.randint(0,y_range)

        img3 = img2.crop((random_x, random_y, width - x_range, height - y_range)).resize((224,224))
        angle = np.random.randint(-5,5)
        img4 = img3.rotate(angle)

        return np.array(img4) / 255, label
    return read_x_y

In [7]:
np.random.shuffle(dataset)

In [8]:
train = dataset[0:int(len(dataset)* .8)]
test = dataset[int(len(dataset)* .8):]

In [9]:
print(len(train))
print(len(test))

10000
2500


In [10]:
batch_size= 100
training_generator = Mygenerator(train,batch_size,wrapper_read_x_y(classes),training = True)
test_generator = Mygenerator(test,batch_size,wrapper_read_x_y(classes), training=False)

In [ ]:
IMG_SHAPE = (224, 224, 3)
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')

In [13]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])


In [14]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(training_generator,validation_data=test_generator,batch_size=batch_size, epochs=1)


In [16]:
model.evaluate(test_generator, verbose=2)

25/25 - 9s - loss: 1.5471 - accuracy: 0.0332 - 9s/epoch - 365ms/step


[1.5471210479736328, 0.03319999948143959]